In [1]:
import os
os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''


In [1]:
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import kagglehub

path = kagglehub.dataset_download("arbethi/diabetic-retinopathy-level-detection")

print("Path to dataset files:", path)

Resuming download from 6663700480 bytes (3704155983 bytes left)...
Resuming download to C:\Users\BALASUBRAMANYAM\.cache\kagglehub\datasets\arbethi\diabetic-retinopathy-level-detection\3.archive (6663700480/10367856463) bytes left.


100%|██████████████████████████████████████████████| 9.66G/9.66G [18:27<00:00, 3.34MB/s]

Extracting files...


Path to dataset files: C:\Users\BALASUBRAMANYAM\.cache\kagglehub\datasets\arbethi\diabetic-retinopathy-level-detection\versions\3


In [2]:
import os

dataset_path = r"C:\Users\BALASUBRAMANYAM\.cache\kagglehub\datasets\arbethi\diabetic-retinopathy-level-detection\versions\3"

os.listdir(dataset_path)


['inception-diabetic.h5', 'preprocessed dataset']

In [3]:
preprocessed_path = os.path.join(dataset_path, "preprocessed dataset")
os.listdir(preprocessed_path)


['preprocessed dataset']

In [4]:
final_path = os.path.join(preprocessed_path, "preprocessed dataset")
os.listdir(final_path)


['testing', 'training']

In [5]:

base_path = r"C:\Users\BALASUBRAMANYAM\.cache\kagglehub\datasets\arbethi\diabetic-retinopathy-level-detection\versions\3"

train_path = os.path.join(base_path, "preprocessed dataset", "preprocessed dataset", "training")
test_path  = os.path.join(base_path, "preprocessed dataset", "preprocessed dataset", "testing")

print("Train Path:", train_path)
print("Test Path:", test_path)


Train Path: C:\Users\BALASUBRAMANYAM\.cache\kagglehub\datasets\arbethi\diabetic-retinopathy-level-detection\versions\3\preprocessed dataset\preprocessed dataset\training
Test Path: C:\Users\BALASUBRAMANYAM\.cache\kagglehub\datasets\arbethi\diabetic-retinopathy-level-detection\versions\3\preprocessed dataset\preprocessed dataset\testing


In [6]:
os.listdir(train_path)


['0', '1', '2', '3', '4']

In [7]:

IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [8]:
imageSize = [299, 299]


In [9]:
xception = Xception(
    input_shape=imageSize + [3],
    weights='imagenet',
    include_top=False
)


In [10]:
for layer in xception.layers:
    layer.trainable = False


In [11]:
x = Flatten()(xception.output)


In [12]:
prediction = Dense(5, activation='softmax')(x)


In [13]:
model = Model(inputs=xception.input, outputs=prediction)


In [14]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)           ┃ Output Shape        ┃     Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer            │ (None, 299, 299, 3) │           0 │ -                    │
│ (InputLayer)           │                     │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block1_conv1 (Conv2D)  │ (None, 149, 149,    │         864 │ input_layer[0][0]    │
│                        │ 32)                 │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block1_conv1_bn        │ (None, 149, 149,    │         128 │ block1_conv1[0][0]   │
│ (BatchNormalization)   │ 32)                 │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block1_conv1_act       │ (None, 149, 149,    │           0 │ block1_conv1_bn[0][… │
│ (Activation)           │ 32)                 │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block1_conv2 (Conv2D)  │ (None, 147, 147,    │      18,432 │ block1_conv1_act[0]… │
│                        │ 64)                 │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block1_conv2_bn        │ (None, 147, 147,    │         256 │ block1_conv2[0][0]   │
│ (BatchNormalization)   │ 64)                 │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block1_conv2_act       │ (None, 147, 147,    │           0 │ block1_conv2_bn[0][… │
│ (Activation)           │ 64)                 │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block2_sepconv1        │ (None, 147, 147,    │       8,768 │ block1_conv2_act[0]… │
│ (SeparableConv2D)      │ 128)                │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block2_sepconv1_bn     │ (None, 147, 147,    │         512 │ block2_sepconv1[0][… │
│ (BatchNormalization)   │ 128)                │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block2_sepconv2_act    │ (None, 147, 147,    │           0 │ block2_sepconv1_bn[… │
│ (Activation)           │ 128)                │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block2_sepconv2        │ (None, 147, 147,    │      17,536 │ block2_sepconv2_act… │
│ (SeparableConv2D)      │ 128)                │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block2_sepconv2_bn     │ (None, 147, 147,    │         512 │ block2_sepconv2[0][… │
│ (BatchNormalization)   │ 128)                │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ conv2d (Conv2D)        │ (None, 74, 74, 128) │       8,192 │ block1_conv2_act[0]… │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ block2_pool            │ (None, 74, 74, 128) │           0 │ block2_sepconv2_bn[… │
│ (MaxPooling2D)         │                     │             │                      │
├────────────────────────┼─────────────────────┼─────────────┼──────────────────────┤
│ batch_normalization    │ (None, 74, 74, 128) │         512 │ conv2d[0][0]         │
│ (BatchNormalization)   │                     │             │                      │
├────────────────────────┼──────────────────

 Total params: 21,885,485 (83.49 MB)

 Trainable params: 1,024,005 (3.91 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [15]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [16]:
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=30
)


Epoch 1/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1538s 13s/step - accuracy: 0.6532 - loss: 4.5006 - val_accuracy: 0.6975 - val_loss: 3.5275
Epoch 2/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1002s 9s/step - accuracy: 0.7043 - loss: 3.7573 - val_accuracy: 0.6812 - val_loss: 3.4231
Epoch 3/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 935s 8s/step - accuracy: 0.7234 - loss: 4.3203 - val_accuracy: 0.6512 - val_loss: 3.6432
Epoch 4/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1014s 8s/step - accuracy: 0.7480 - loss: 3.5281 - val_accuracy: 0.7180 - val_loss: 2.7742
Epoch 5/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 962s 8s/step - accuracy: 0.7662 - loss: 2.9897 - val_accuracy: 0.7902 - val_loss: 2.4154
Epoch 6/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1115s 10s/step - accuracy: 0.7605 - loss: 3.3977 - val_accuracy: 0.7371 - val_loss: 2.5170
Epoch 7/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 966s 8s/step - accuracy: 0.7777 - loss: 3.0317 - val_accuracy: 0.7071 - val_loss: 3.5126
Epoch 8/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 943s 8s/step - accuracy: 0.7613 - loss: 3.6175 - va

In [20]:
model.save("model/Updated-Xception-diabetic-retinopathy.h5")


In [19]:
# import os
# print(os.listdir("model"))
import os
print(os.listdir())


['.git', '.idea', '.ipynb_checkpoints', 'app.py', 'dataset', 'Early Detection of Diabetic Retinopathy.docx', 'LICENSE', 'note.ipynb', 'README.md', 'templates', 'Updated-Xception-diabetic-retinopathy.h5']


In [21]:
import os

os.makedirs("model", exist_ok=True)
print("model folder created ✅")


model folder created ✅


In [22]:
import shutil

shutil.move(
    "Updated-Xception-diabetic-retinopathy.h5",
    "model/Updated-Xception-diabetic-retinopathy.h5"
)

print("Model moved successfully ✅")


Model moved successfully ✅


In [23]:
print(os.listdir("model"))


['Updated-Xception-diabetic-retinopathy.h5']
